# Рекомендация тарифов

Даны данные о поведении клиентов, которые уже перешли на определенные тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф для клиента.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
<ul>
<li>сalls — количество звонков</li>
<li>minutes — суммарная длительность звонков в минутах</li>
<li>messages — количество sms-сообщений</li>
<li>mb_used — израсходованный интернет-трафик в Мб</li>
<li>is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)</li>
</ul>

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('users_behavior.csv')
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<b>Вывод:</b> как и сказано в условии, в данные уже очищены от пропусков. Столбце с тарифом (is_ultra) качественный, то есть будем решать задачу классификации.

## Разбейте данные на выборки

Требуется разбить данные на обучающую, валидационную и тестовую выборку. Размер валидационной и тестовой выборок обычно равны. Будем использовать соотношение 3:1:1, где 3 части приходится на обучающую выборку и по 1 на тестовую и валидационную.

Для начала разделим данные на обучающую выборку и выборку, в которую будут входить валидационные и тестовые данные. В этом случае вторая выборка составит 40%. А далее вторую выборки разделим на валидационную и тестовую.

In [4]:
features = data.drop('is_ultra', axis = 1)
target = data['is_ultra']
features_train, data_valid_test_features, target_train, data_valid_test_target= train_test_split(features, target, test_size = 0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(data_valid_test_features, data_valid_test_target, test_size = 0.5, random_state = 12345)
print('Размер всех данных:', data.shape)
print('Размер обучающей выборки (без целевого признака):', features_train.shape)
print('Размер валидационной выборки (без целевого признака):', features_valid.shape)
print('Размер тестовой выборки (без целевого признака):', features_test.shape)

Размер всех данных: (3214, 5)
Размер обучающей выборки (без целевого признака): (1928, 4)
Размер валидационной выборки (без целевого признака): (643, 4)
Размер тестовой выборки (без целевого признака): (643, 4)


<b>Вывод:</b> данные разделены на 3 выборки: обучающую(60%), валидационную(20%) и тестовую(20%). А их в свою очередь разделили на признаки и целевой признак.

## Исследуйте модели

Будем проверять как работают 3 модели: решающее дерево с различной глубиной, случайный лес с различным количеством деревьев и глубиной дерева, а также логистическую регрессию.

### Алгоритм решающего дерева

In [5]:
max_accuracy = 0.7
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > max_accuracy:
        best_depth = depth
        max_accuracy = accuracy
print('max_depth = {} : {}'.format(best_depth, max_accuracy))

max_depth = 3 : 0.7853810264385692


При решающем дереве максимальная доля правильных ответов получается при глубине дерева 3 и равна 0.7853810264385692

### Алгоритм случайного леса

In [6]:
best_model = None
best_result = 0.7
best_est = 0
best_depth = 0
for est in range(5, 51, 5):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy_valid = accuracy_score(target_valid, predictions_valid)
        if accuracy_valid > best_result:
            best_model = model
            best_result = accuracy_valid
            best_est = est
            best_depth = depth

print("Доля правильных ответов наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Доля правильных ответов наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 10


В случае с рандомным лесом лучший accuracy получился при количестве деревьев 40 и глубиной 10. Доля правильных ответов в данном случае равна 0.8087091757387247. Но в данной модели минус в том, что работает она гораздо медленнее, чем решающее дерево.

### Алгоритм логистической регрессии

In [7]:
model_regression = LogisticRegression(random_state = 12345)
model_regression.fit(features_train, target_train) 
result = model_regression.score(features_valid, target_valid)
print(result)

0.7107309486780715


В случае в логистической легрессией получаем долю правильных ответов 0.7107309486780715, что ниже, чем в случае со случайным лесом.

<b>Вывод</b>: в данном случае лучший результат дал алгоритм случайноого леса с 40 деревьями и глубиной дерева 10. Доля правильных ответов при этом составляет 0.8087091757387247. Остальные 2 алгоритма дают результаты ниже. Используем модель случайного леса для проверки на тестовой выборке.

## Проверьте модель на тестовой выборке

In [8]:
predictions_test = best_model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
print(accuracy_test)

0.7962674961119751


<b>Вывод:</b> проверили тестовую выборку на лучшей модели и доля правильных ответов в тестовой выборке составила 0.7962674961119751.

## Проверьте модели на адекватность

Для проверки на адекватность можно использовать алгоритм DummyClassifier, из библиотеки sklearn.dummy, которые покажет предсказание модели использую простые правила. Этот алгоритм не используется для реальных проблем, а создан для сравнения с реальной моделью. Используем данную модель со стратегией most_frequent.

In [9]:
frequent_model = DummyClassifier(strategy = 'most_frequent').fit(features_train, target_train)
predictions_frequent = frequent_model.predict(features_valid)
print(frequent_model.score(features_valid, target_valid))

0.7060653188180405


В данном алгоритме доля правильных ответов получилась меньше, чем в выбранной модели. Это говорит о том, что выбранный алгоритм адекватен.

<b>Вывод:</b> проверив наши данные на модели, которая не используется в реальных заданиях, а создана лишь для проверки реальных моделей, можно сказать, что выбранная нами модель случайного леса дает большую долю правильных ответов.

## Вывод

Для данных были построены 3 модели: решающего дерева, случайного леса и логистической регрессии. Самой лучшей моделью оказался случайный лес с долей правильных ответов 0.8087091757387247 с количеством деревьев 40 и глубиной дерева 10. Модель была проверена на адекватность. На тестовых данных модель показала долю правильных ответов 0.7962674961119751, то есть с вероятностью почти 0.8 можно будет по количеству звонков, суммарной длительности звонков в минутах, количеству sms-сообщений и израсходованному интернет-трафику в Мб понять какой тариф следует предложить таму или иному абоненту.